### Dataset by:

@misc{rozemberczki2021twitch,
      title={Twitch Gamers: a Dataset for Evaluating Proximity Preserving and Structural Role-based Node Embeddings}, 
      author={Benedek Rozemberczki and Rik Sarkar},
      year={2021},
      eprint={2101.03091},
      archivePrefix={arXiv},
      primaryClass={cs.SI}
}


### Social Network Analysis of Twitch Partners
A social network of Twitch users which was collected from the public API in Spring 2018. Nodes are Twitch users and edges are mutual follower relationships between them. The graph forms a single strongly connected component without missing attributes. The machine learning tasks related to the graph are count data regression and node classification.

In [1]:
import networkx as nx   
import pandas as pd

In [2]:
fh = open("data/large_twitch_edges.csv", "rb")

G = nx.read_edgelist(fh, delimiter=',')

fh.close()

In [3]:
G['1']

AtlasView({'113417': {}, '95914': {}, '155127': {}, '71050': {}, '112881': {}, '57532': {}, '64605': {}, '8079': {}, '6250': {}, '125642': {}, '30268': {}, '53724': {}, '123076': {}, '165361': {}, '149069': {}, '42688': {}, '90650': {}, '35358': {}, '8176': {}, '54063': {}, '32920': {}, '31140': {}, '56352': {}, '83691': {}, '87271': {}, '32338': {}, '61862': {}, '37818': {}, '55192': {}, '85756': {}, '110345': {}, '85701': {}, '52703': {}, '119034': {}, '152296': {}, '144643': {}, '161362': {}, '108809': {}, '128864': {}, '35408': {}, '94483': {}, '83095': {}, '61780': {}, '78367': {}, '26309': {}, '86321': {}, '119017': {}, '79844': {}, '89069': {}, '158403': {}, '78354': {}, '39372': {}, '58493': {}, '126773': {}, '128328': {}, '130232': {}, '105992': {}, '5507': {}, '64419': {}, '55261': {}, '45001': {}, '120819': {}, '98175': {}, '53444': {}, '38031': {}, '34625': {}, '41855': {}, '86266': {}, '137899': {}, '21357': {}, '40868': {}, '148905': {}, '117105': {}, '67333': {}, '8121':

In [4]:
features = pd.read_csv('data/large_twitch_features.csv')


In [5]:
features

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016-02-16,2018-10-12,0,0,EN,1
1,500,0,2699,2011-05-19,2018-10-08,1,0,EN,0
2,382502,1,3149,2010-02-27,2018-10-12,2,0,EN,1
3,386,0,1344,2015-01-26,2018-10-01,3,0,EN,0
4,2486,0,1784,2013-11-22,2018-10-11,4,0,EN,0
...,...,...,...,...,...,...,...,...,...
168109,4965,0,810,2016-07-20,2018-10-08,168109,0,EN,0
168110,4128,1,2080,2013-01-31,2018-10-12,168110,0,EN,0
168111,3545,0,1797,2013-11-08,2018-10-10,168111,0,EN,1
168112,892736,1,2135,2012-12-07,2018-10-12,168112,0,EN,0


In [6]:
node_list = []
for rows in features.iterrows():
    node_list.append((str(rows[1]['numeric_id']), {'views': rows[1]['views']}))

In [7]:
node_list

[('0', {'views': 7879}),
 ('1', {'views': 500}),
 ('2', {'views': 382502}),
 ('3', {'views': 386}),
 ('4', {'views': 2486}),
 ('5', {'views': 4987}),
 ('6', {'views': 234}),
 ('7', {'views': 775}),
 ('8', {'views': 69020}),
 ('9', {'views': 32073}),
 ('10', {'views': 94}),
 ('11', {'views': 10254468}),
 ('12', {'views': 2178}),
 ('13', {'views': 4871}),
 ('14', {'views': 2002}),
 ('15', {'views': 33882}),
 ('16', {'views': 200}),
 ('17', {'views': 11624}),
 ('18', {'views': 1160}),
 ('19', {'views': 144812}),
 ('20', {'views': 1885}),
 ('21', {'views': 52160}),
 ('22', {'views': 212680}),
 ('23', {'views': 14538}),
 ('24', {'views': 339218}),
 ('25', {'views': 1025}),
 ('26', {'views': 9528}),
 ('27', {'views': 2802}),
 ('28', {'views': 162078}),
 ('29', {'views': 2585}),
 ('30', {'views': 1370}),
 ('31', {'views': 4405}),
 ('32', {'views': 21502}),
 ('33', {'views': 745}),
 ('34', {'views': 17042}),
 ('35', {'views': 2559}),
 ('36', {'views': 5950}),
 ('37', {'views': 8231}),
 ('38', 

In [8]:
G.add_nodes_from(node_list)

In [13]:
list(G.adj['1'])

['113417',
 '95914',
 '155127',
 '71050',
 '112881',
 '57532',
 '64605',
 '8079',
 '6250',
 '125642',
 '30268',
 '53724',
 '123076',
 '165361',
 '149069',
 '42688',
 '90650',
 '35358',
 '8176',
 '54063',
 '32920',
 '31140',
 '56352',
 '83691',
 '87271',
 '32338',
 '61862',
 '37818',
 '55192',
 '85756',
 '110345',
 '85701',
 '52703',
 '119034',
 '152296',
 '144643',
 '161362',
 '108809',
 '128864',
 '35408',
 '94483',
 '83095',
 '61780',
 '78367',
 '26309',
 '86321',
 '119017',
 '79844',
 '89069',
 '158403',
 '78354',
 '39372',
 '58493',
 '126773',
 '128328',
 '130232',
 '105992',
 '5507',
 '64419',
 '55261',
 '45001',
 '120819',
 '98175',
 '53444',
 '38031',
 '34625',
 '41855',
 '86266',
 '137899',
 '21357',
 '40868',
 '148905',
 '117105',
 '67333',
 '8121',
 '31559',
 '85877',
 '111372',
 '107854',
 '161658',
 '14649',
 '77138',
 '11288',
 '11604',
 '84226',
 '146335',
 '37297',
 '139448',
 '113692',
 '165834',
 '149295',
 '96173',
 '32147',
 '1852',
 '146235',
 '88518',
 '145545',
 '

In [14]:
G.nodes['1']

{'views': 500}